In [1]:
# desc

In [2]:
# import the smartpynector module
from NBs import smartpynector as sp


In [3]:
# Defining constants
# Knowledge engine REST API URL
KNOWLEDGE_ENGINE_URL = "http://127.0.0.1:8280/rest"
THERMOSTAT_ID = "http://127.0.0.1:8000/thermostat"

In [4]:
# Defining the Smart Connector
smart_connector = sp.SmartConnector(
    knowledgeBaseId=THERMOSTAT_ID,
    knowledgeBaseName="thermostat 1",
    knowledgeBaseDescription="thermostat 1",
    reasonerEnabled=False,
)

In [5]:
sp.create_smart_connector(smart_connector, KNOWLEDGE_ENGINE_URL)

2023-03-08 23:57:30 DEBUG HTTP Request: POST http://127.0.0.1:8280/rest/sc "HTTP/1.1 200 OK"
2023-03-08 23:57:30 INFO Request successful
2023-03-08 23:57:30 DEBUG 


<Response [200 OK]>

In [6]:
answer_ki_id = sp.register_knowledge_interaction(
   knowledge_interaction_type="AnswerKnowledgeInteraction",
   knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
   prefixes={
       "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
       "saref": "https://w3id.org/saref#",
   },
   graph_pattern='''?meas rdf:type saref:Measurement .
         ?meas saref:hasValue ?temp .
         ?meas saref:isMeasuredIn saref:TemperatureUnit .
         ?meas saref:hasTimestamp ?timestamp .
         ?meas saref:isMeasurementOf ?room_id .
         ?meas saref:relatesToProperty saref:Temperature .
         ?meas saref:measurementMadeBy ?device_id .''',
   headers={
        "Content-Type": "application/json",
        "knowledge-Base-Id": THERMOSTAT_ID
   },
).json()["knowledgeInteractionId"]

answer_ki_id

2023-03-08 23:57:30 DEBUG HTTP Request: POST http://127.0.0.1:8280/rest/sc/ki "HTTP/1.1 200 OK"
2023-03-08 23:57:30 INFO Request successful
2023-03-08 23:57:30 DEBUG {"knowledgeInteractionId":"http://127.0.0.1:8000/thermostat/interaction/0adb2331-fc78-4db0-91f0-a8a29c3d002b"}


'http://127.0.0.1:8000/thermostat/interaction/0adb2331-fc78-4db0-91f0-a8a29c3d002b'